In [1]:
import sympy
import numpy as np

# Linewidths from coherence and power 
The general form of the Master equation is given by
$$
\frac{\partial \hat{\rho}}{\partial t} = -\frac{i}{\hbar}\left[\hat{H},\hat{\rho}\right]+\sum_i\gamma_i\left(\hat{c}\hat{\rho}\hat{c}^{\dagger}-\frac{1}{2}\left(\hat{c}^{\dagger}\hat{c}\hat{\rho}+\hat{\rho}\hat{c}^{\dagger}\hat{c}\right)\right)
$$
We rewrite this stuff as

$$
\frac{\partial \vec{\rho}}{\partial t} = \hat{\mathcal{L}}\vec{\rho},
$$
with the lindblad superoperator $\hat{\mathcal{L}}$ given by
$$\hat{\mathcal{L}} = 
    \left(-\frac{i}{\hbar}
        \left(
            \hat{I}\otimes\hat{H}^T-
            \hat{H}\otimes\hat{I}
        \right)
   +\sum\limits_i\left(
       \hat{C}_i^*\otimes\hat{C}-
       \frac{1}{2}\left(\hat{C}\hat{C}^T\otimes\hat{I}+
       \hat{I}\otimes\hat{C}^T\hat{C}\right)
   \right)
\right) 
$$
Important: maybe I missed a conjugation or two.

## Defining some helper functions to convert \hat{H} and \hat{C} to corresponding Lindblad superoperator parts.

In [8]:
def H2Liou(H):
    L1 = np.kron(np.identity(H.shape[0]), H.T)
    L2 = np.kron(H, np.identity(H.shape[0]))
    return L2-L1
def C2LSO(C):
    CC = C.T@np.conj(C)
    CC1 = np.kron(np.identity(CC.shape[0]), CC.T)
    CC2 = np.kron(CC, np.identity(CC.shape[0]))
    CC3 = np.kron(np.conj(C), C)
    return CC3-(CC1+CC2)/2

## defining symbols, Hamiltonian, collapse operators

In [11]:
Δ, Ω, γ1, γϕ = sympy.symbols('Δ, Ω, γ_1, γ_ϕ', real=True, positive=True)
H = sympy.Matrix([[0, Ω/2],[Ω/2, Δ]])
C_loss = sympy.Matrix([[0, sympy.sqrt(γ1)],[0, 0]])
C_dephasing = sympy.Matrix([[sympy.sqrt(γϕ/2),0],[0,-sympy.sqrt(γϕ/2)]])


Liou = sympy.nsimplify(H2Liou(H))
Lind_loss = sympy.nsimplify(C2LSO(C_loss))
Lind_dephasing = sympy.nsimplify(C2LSO(C_dephasing))

Lindbladian = (-sympy.I*Liou+Lind_loss+Lind_dephasing).tomatrix()
#Lindbladian = Lind_loss.tomatrix()
Lindbladian

Matrix([
[     0,             I*Ω/2,             -I*Ω/2,    γ_1],
[ I*Ω/2, I*Δ - γ_1/2 - γ_ϕ,                  0, -I*Ω/2],
[-I*Ω/2,                 0, -I*Δ - γ_1/2 - γ_ϕ,  I*Ω/2],
[     0,            -I*Ω/2,              I*Ω/2,   -γ_1]])

## Transform to go to Pauli basis

In [12]:
# transform to Pauli
si = [[1,0],[0,1]]
sx = [[0,1],[1,0]]
sy = [[0,-sympy.I],[sympy.I,0]]
sz = [[1,0],[0,-1]]
nq = 1
import itertools
T = np.zeros(Lindbladian.shape, dtype=object)
for op_id, Π in enumerate(itertools.product(*([[si, sx, sy, sz]]*nq))):
    T1 = [[1]]
    for Π1 in Π:
        T1 = np.kron(T1, Π1)
    T[op_id, :] = np.conj(T1.ravel())
T = sympy.nsimplify(sympy.Matrix(T))
T

Matrix([
[1, 0,  0,  1],
[0, 1,  1,  0],
[0, I, -I,  0],
[1, 0,  0, -1]])

In [15]:
Bloch_eq = sympy.simplify(T@Lindbladian@T.inv())
Bloch_eq

Matrix([
[  0,            0,            0,    0],
[  0, -γ_1/2 - γ_ϕ,            Δ,    0],
[  0,           -Δ, -γ_1/2 - γ_ϕ,   -Ω],
[γ_1,            0,            Ω, -γ_1]])

## Finding stationary state from nullspace of Bloch equations

In [16]:
γ2 = sympy.Symbol('γ_2', real=True, positive=True)
stationary_state_bloch = sympy.simplify(sympy.simplify(Bloch_eq.nullspace()[0]).subs({(γ1+2*γϕ):sympy.nsimplify(2)*γ2}))
stationary_state_bloch = stationary_state_bloch/stationary_state_bloch[0]
stationary_state_bloch

Matrix([
[                                                   1],
[           -Δ*Ω*γ_1/(Ω**2*γ_2 + γ_1*(Δ**2 + γ_2**2))],
[         -Ω*γ_1*γ_2/(Ω**2*γ_2 + γ_1*(Δ**2 + γ_2**2))],
[γ_1*(Δ**2 + γ_2**2)/(Ω**2*γ_2 + γ_1*(Δ**2 + γ_2**2))]])

## FWHM from the z-projection of the Bloch vector

In [18]:
# spectrum FWHM
n_ex = (-stationary_state_bloch[3]+1)/2
sympy.simplify(n_ex)
n_ex_max = n_ex.subs({Δ:0})
solutions = sympy.solveset(n_ex-n_ex_max/2, Δ)
FWHM = max(solutions)-min(solutions)
FWHM 

2*sqrt(γ_2)*sqrt(Ω**2 + γ_1*γ_2)/sqrt(γ_1)